<a href="https://colab.research.google.com/github/zzl120/mnist_tutorial/blob/master/stylegan2_ada_pytorch_pokemon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Pokemon with StyleGAN

In this tutorial, we only want to familiarize ourselves with running StyleGAN (in this case, the newest version) and seeing how training progresses.

## Verify Runtime is GPU

In the menu, select Runtime -> Change Runtime Type and verify you are using the **GPU**.  Also select **High-RAM** if you are using Colab Pro.

The `nvidia-smi` command below should **NOT** display *"NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver."*

In [24]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-34d433b7-1e41-f0d2-aeba-227ccd35c8d7)


## Mount your Google Drive

You will be storing the training models and progress images on your Google Drive.  This is very convenient for viewing progress, and if your Colab notebook is disconnected you will not lose your models.

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Install Stylegan2-ada-pytorch Prerequisites

In [26]:
import torch

Verify the next command results in "1".  If not, go back to the beginning and verify you have a GPU runtime.

In [27]:
torch.cuda.device_count()

1

In [28]:
import torchvision

In [29]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

In [30]:
!pip install ninja opensimplex torch==1.7.1 torchvision==0.8.2

## Get the StyleGAN code

In [7]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 4.80 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [8]:
mkdir /content/stylegan2-ada-pytorch/datasets

## Get Pokemon dataset of images

In [9]:
cd /content/stylegan2-ada-pytorch/datasets

/content/stylegan2-ada-pytorch/datasets


In [10]:
ls

In [11]:
!wget https://a.y8j5.top/s/MW25lFJ

--2022-02-20 09:16:54--  https://a.y8j5.top/s/MW25lFJ
Resolving a.y8j5.top (a.y8j5.top)... 163.181.33.226, 163.181.33.227, 163.181.33.228, ...
Connecting to a.y8j5.top (a.y8j5.top)|163.181.33.226|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://xmyp28-my.sharepoint.cn/:u:/g/personal/admin_xmyp28_partner_onmschina_cn/ETeg6BsKohJJo1n2CWRSXMwB8g-FGcqGqgOdqKBs5buKpg?download=1 [following]
--2022-02-20 09:16:58--  https://xmyp28-my.sharepoint.cn/:u:/g/personal/admin_xmyp28_partner_onmschina_cn/ETeg6BsKohJJo1n2CWRSXMwB8g-FGcqGqgOdqKBs5buKpg?download=1
Resolving xmyp28-my.sharepoint.cn (xmyp28-my.sharepoint.cn)... 40.73.129.53, 2406:e500:4600:1::53
Connecting to xmyp28-my.sharepoint.cn (xmyp28-my.sharepoint.cn)|40.73.129.53|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/admin_xmyp28_partner_onmschina_cn/Documents/2022/2/20/13499d9d-f12a-4047-bd61-c0e40ebe1619/custom.zip [following]
--2022-02-20 09:16:5

In [13]:
!unzip MW25lFJ

Archive:  MW25lFJ
   creating: custom/
  inflating: custom/00013edfe31d2ce237ee1c6df676e5a3.png  
  inflating: custom/0012b9a4cc7fac34bd2c3ea9f93961a4.png  
  inflating: custom/002a4dfc2199b01eabb1ffcb58f1ed29.png  
  inflating: custom/002d853a7272ce55fb368c69757a3acc.png  
  inflating: custom/005377f5f29e5585fe5e9cc0788a0ae9.png  
  inflating: custom/008d12c81e507113107b1908c09bb9e9.png  
  inflating: custom/00db48bce048863477b0d4e88e099ecb.png  
  inflating: custom/010354469223b993f1d0dbf2a78d765a.png  
  inflating: custom/010a48affaa193411134fa76eaa2edfe.png  
  inflating: custom/012f4c8e0520c0b712404d7d2b10fde2.png  
  inflating: custom/0131becdd42eba726f6f85e67cc42008.png  
  inflating: custom/013f2fd09843d0a0623ba7d527b56f54.png  
  inflating: custom/01515053ab90bee825b4cd719b5825af.png  
  inflating: custom/0163735960550dd9ba05bc73aeb571db.png  
  inflating: custom/0164912b9cbf61c2de8c30fd7962855c.png  
  inflating: custom/017f5ecf1cde5977d062fa1556e7c694.png  
  inflating: cust

In [ ]:
!rm -rf __MACOSX

In [ ]:
!rm pokemon256.zip

/content/stylegan2-ada-pytorch


In [15]:
!wget https://a.y8j5.top/s/Y1ygdSP

--2022-02-20 09:17:45--  https://a.y8j5.top/s/Y1ygdSP
Resolving a.y8j5.top (a.y8j5.top)... 163.181.33.226, 163.181.33.227, 163.181.33.228, ...
Connecting to a.y8j5.top (a.y8j5.top)|163.181.33.226|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://xmyp22-my.sharepoint.cn/:u:/g/personal/admin_xmyp22_partner_onmschina_cn/EXtFTJs7fXRFtOhngp6kGmMBxEoaZmHF0toKJUTvWEWDZg?download=1 [following]
--2022-02-20 09:17:48--  https://xmyp22-my.sharepoint.cn/:u:/g/personal/admin_xmyp22_partner_onmschina_cn/EXtFTJs7fXRFtOhngp6kGmMBxEoaZmHF0toKJUTvWEWDZg?download=1
Resolving xmyp22-my.sharepoint.cn (xmyp22-my.sharepoint.cn)... 40.73.129.53, 2406:e500:4600:1::53
Connecting to xmyp22-my.sharepoint.cn (xmyp22-my.sharepoint.cn)|40.73.129.53|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/admin_xmyp22_partner_onmschina_cn/Documents/2022/2/20/f3a79147-4a6d-46a5-909e-0880b19b0b4e/network-snapshot-000028.pkl [following]
--2

In [16]:
mv Y1ygdSP network-snapshot-000028.pkl

In [17]:
ls

calc_metrics.py*  docker_run.sh*  metrics/                     torch_utils/
datasets/         docs/           network-snapshot-000028.pkl  training/
dataset_tool.py*  generate.py*    projector.py*                train.py*
dnnlib/           legacy.py*      README.md*
Dockerfile*       LICENSE.txt*    style_mixing.py*


## Prepare Pokemon dataset for use by StyleGAN

In [18]:
!python dataset_tool.py --source=./datasets/custom --dest=./datasets/custom.zip

100% 2011/2011 [00:10<00:00, 192.83it/s]


## Create Folders on Your Google Drive

If we accidentally close our browser or the Colab runtime disconnects, we will lose all of our training models and progress images.  Therefore we want to store the training data on our Google Drive.  The following cells will create a new folder on your Google Drive, **MachineLearningForArtists**.  The Pokemon training data will be stored in **MachineLearningForArtists/Pokemon**

In [19]:
mkdir /content/drive/MyDrive/MachineLearningForArtists

mkdir: cannot create directory ‘/content/drive/MyDrive/MachineLearningForArtists’: File exists


In [20]:
mkdir /content/drive/MyDrive/MachineLearningForArtists/Monkey

mkdir: cannot create directory ‘/content/drive/MyDrive/MachineLearningForArtists/Monkey’: File exists


If you get an error "cannot create directory", it's probably because the folder already exists on your Google Drive and you can ignore the error.

## Train from Scratch

This cell block will train StyleGAN from scratch.  Training from scratch is much slower than using *transfer learning* on a previously trained model.  However, the purpose of this tutorial is to a) try training for the first time and b) notice how the progress images develop over time.

In [ ]:
!python train.py --outdir=/content/drive/MyDrive/MachineLearningForArtists/Monkey --data=./datasets/custom.zip --gpus=1 --augpipe=bg --gamma=10 --cfg=paper256 --mirror=1 --snap=5 --metrics=none


## Wait an Hour or Two

Normal training of a dataset can take days.  For this demonstration, we only want to train for a few hours in order to get familiar with the training process and see the initial results.

Results are stored in your Google Drive under drive/MyDrive/MachineLearningForArtists/Pokemon

Each training run is stored in a separate directory.  The initial training run is stored in `00000-pokemondataset...`.  If you run training twice, the second run will be stored in `00001-pokemondataset...`.  And so on.

Inside the training run directory you will see various files.  `reals.png` shows a sample of the training dataset.  You should see various Pokemon in this image. `fakes000000.png` is a sample of generated images from the initial model.  `network-snapshot-XXXXXX.pkl` is the actual model which can be used later to generate "fake" images, videos, etc.

As training progresses you will see more `fakes` & `network-snapshot` files created.

Since we only want to test a training, once you get to around the 500th iteration, you can stop the above training cell.  Then take a look at the `fakes` images and familiarize yourself with how the training progresses.



## Transfer Learning

Now, let's train the same dataset but use *transfer learning*.  We will use a StyleGAN2 model already pre-trained on Pokemon to train a new model that is trained against a Tamagotchi dataset.

### Get the Tomagotchi dataset

In [ ]:
%cd /content/stylegan2-ada-pytorch/datasets

In [ ]:
!wget https://github.com/derekphilipau/machinelearningforartists/raw/main/docs/src/week2/Tamagotchi256.zip

In [ ]:
!unzip Tamagotchi256.zip

In [ ]:
!rm -rf __MACOSX/

In [ ]:
%cd /content/stylegan2-ada-pytorch

### Prepare Tomagotchi dataset for use by StyleGAN

In [ ]:
!python dataset_tool.py --source=./datasets/Tamagotchi256 --dest=./datasets/tamagotchidataset.zip

### Get the pre-trained Pokemon model

In [ ]:
!gdown --id 122_Nu7p6ESMqBarxuAHCXhQuAaNCsh_a

### Train using the Pokemon model with the Tamagotchi dataset 

In [ ]:
!python train.py --resume=./network-snapshot-000028.pkl --outdir=/content/drive/MyDrive/MachineLearningForArtists/Monkey --data=./datasets/custom.zip --gpus=1 --augpipe=bg --gamma=10 --cfg=paper256 --mirror=1 --snap=1 --metrics=none



Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 1,
  "network_snapshot_ticks": 1,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "./datasets/custom.zip",
    "use_labels": false,
    "max_size": 2011,
    "xflip": true,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 8
    },
    "channel_base": 16384,
    "channel_max": 512,
    "num_fp16_res": 4,
 